In [2]:
from pyscenic.rnkdb import FeatherRankingDatabase
import os

In [3]:
DB_FOLDER = "/Users/bramvandesande/Projects/lcb/databases"

In [5]:
db = FeatherRankingDatabase(fname=os.path.join(DB_FOLDER, "hg19-regions-220330-9species.extracted.feather"),
                            name="regions", nomenclature="HGNC")

In [7]:
len(db.genes)

220329

In [9]:
df = db.load_full()

In [10]:
df.head()

,chr1-reg100012,chr1-reg100013,chr1-reg100015,chr1-reg100017,chr1-reg100393,chr1-reg100395,chr1-reg100396,chr1-reg100397,chr1-reg100398,chr1-reg100399,...,chrY-reg821,chrY-reg822,chrY-reg824,chrY-reg858,chrY-reg860,chrY-reg932,chrY-reg934,chrY-reg936,chrY-reg954,chrY-reg956
features,,,,,,,,,,,,,,,,,,,,,
elemento-AAAATGGCG,75041,66521,170298,89191,15955,133523,125421,181496,55105,32557,...,198193,199717,4074,11486,201013,206084,203131,206316,132518,35827
elemento-AAATCAAT,128076,60909,15842,129035,3075,154370,113107,54530,88071,121245,...,205406,110699,134370,45716,208067,120090,196344,196051,135376,54483
elemento-AAATGCAAA,125295,83068,8941,166272,3386,48561,56616,28609,38011,192740,...,205249,201256,14365,67047,206071,209356,199034,208552,131312,2613
elemento-AAATTGCA,16771,90937,3582,29068,35427,22741,74436,18174,117520,87662,...,209474,152863,117874,26385,209573,213381,184826,196811,136602,757
elemento-AACAGCTG,103754,44631,9151,171969,23342,118882,94157,88738,212092,197285,...,76770,199852,73694,36156,208168,216507,197211,190880,135788,94576


In [14]:
len(df)

9713

In [30]:
df.memory_usage(deep=True).sum()

8560995038

In [34]:
nr_chars = max(map(len,df_.columns))
nr_chars

14

In [39]:
import numpy as np
import pandas as pd
df_ = df.head()
nr_chars = max(map(len,df_.columns))
TOP_N = 50000
n_features = len(df_)
inverted_data = np.empty(shape=(n_features, TOP_N), dtype='U{}'.format(nr_chars))
for idx, (_, row) in enumerate(df_.iterrows()):
    inverted_data[idx, :] = row.sort_values(ascending=True).head(TOP_N).index
inverted_df = pd.DataFrame(data=inverted_data, index=df_.index, columns=list(range(TOP_N)))

In [40]:
inverted_df

,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
features,,,,,,,,,,,,,,,,,,,,,
elemento-AAAATGGCG,chrX-reg32698,chr3-reg11533,chr2-reg79638,chr3-reg110064,chr1-reg108107,chr7-reg58012,chr16-reg9028,chr11-reg40102,chr11-reg8174,chr11-reg9500,...,chr8-reg23479,chr16-reg45922,chr19-reg8404,chr12-reg77896,chr1-reg18112,chr1-reg28342,chr12-reg7113,chr5-reg8302,chr16-reg44441,chr17-reg70760
elemento-AAATCAAT,chr10-reg33634,chr9-reg32450,chr7-reg19391,chr2-reg80602,chr6-reg57419,chr13-reg43061,chr17-reg42372,chr14-reg3151,chr18-reg35521,chr14-reg50228,...,chr12-reg86983,chr7-reg70627,chr11-reg66224,chr22-reg4498,chr11-reg4974,chr15-reg9512,chr1-reg131822,chr7-reg62196,chr2-reg42740,chr17-reg33927
elemento-AAATGCAAA,chr17-reg60295,chr1-reg165040,chr10-reg66722,chr10-reg34533,chr7-reg2132,chr11-reg81814,chr20-reg16942,chr9-reg57882,chr11-reg91095,chr5-reg91502,...,chr17-reg635,chr1-reg42975,chr8-reg27484,chr11-reg52580,chr1-reg102464,chr11-reg73235,chr15-reg26297,chr5-reg112328,chr1-reg44410,chr21-reg22019
elemento-AAATTGCA,chr5-reg85184,chr17-reg33458,chr2-reg118522,chr10-reg94934,chr1-reg78218,chr2-reg116500,chr10-reg34533,chrX-reg32938,chrX-reg49548,chr1-reg89950,...,chr8-reg62041,chrX-reg47193,chr5-reg20272,chrX-reg5653,chr18-reg2385,chr19-reg22852,chr1-reg56518,chr9-reg66776,chr4-reg61821,chr14-reg15767
elemento-AACAGCTG,chr2-reg144559,chr2-reg114576,chr2-reg122253,chr11-reg61192,chr3-reg100749,chr9-reg28130,chr14-reg31649,chr15-reg7899,chr4-reg80579,chr10-reg9206,...,chr14-reg18607,chr20-reg48928,chr11-reg16370,chr14-reg59120,chr22-reg25707,chr20-reg5231,chr1-reg109841,chrX-reg11221,chr3-reg75807,chr2-reg103367


In [41]:
inverted_df.memory_usage(deep=True).sum()

17612575